# Interactive Cropping Tool Demo

In [24]:
# import interactive_crop
import os
image_list = ['image_samples/'+i for i in os.listdir('image_samples')]
main(image_list)

GridBox(children=(Dropdown(description='Image Name:', layout=Layout(grid_area='im_selector'), options=('image_…

In [25]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, Layout, GridBox, fixed
from IPython.display import display, HTML, clear_output
import time
from PIL import Image, ImageDraw
from sys import exit as sysexit
from shapes import Rectangle, Ellipse
import numpy as np

def convert_sizes(x_size, y_size):
    """
    Converts the x_size, a 2-elem tuple obtained from the x_slider, and the y_size, 
    a 2-elem tuple obtained from the y-slider, into the size format for the shape of interest.
    """

def main(image_list, crop_shape = 'Rectangle', continuous_update=True, optimize = True,
    callback = lambda x,y: print('{}: {}'.format(x,y))):
    """
    This function takes a list of images and allows the user interactively crop these images through a vertical range slider and a horizontal range slider. Once the crop size is accepted, the callback kwarg will be called and provided the name of the cropped image (from image_list) and the shape of the cropped_image.

    Arguments: 
    image_list - A list of image names
    
    shape - The desired type of crop shape
    continuous_update (default: True) - Bool to indicate whether the plot should dynamically re-render as the user drags the slider, or should wait til slider release to rerender
    optimize (default: True)- Bool to indicate whether the program can optimize for faster rendering of crop boxes, including displaying grayscale images
    outline_color (default: red) - 
    callback - The function to call after the crop size is accepted (via button click)

    Returns: 
    coord_dict - A dict mapping image names to cropped coordinates
    """
    # Need these to retain the global scope
    coord_dict = {}
    SHAPE_DICT = {'Rectangle':Rectangle,'Ellipse':Ellipse} # shape arg mapping
    
    def show_image(image_name):
        """
        Shows the image at the desired size
        """
        clear_output()
        global im
        global im_draw
        # Read image from file
        im = Image.open(image_name)
        if optimize:
            im = im.convert('L')
            im = Image.fromarray(np.array(im).astype(np.uint8))
        
        # Create copy of image to draw on
        im_draw =im.copy()
        display(HTML('<h3 style="margin:5px;text-align:center">'+image_name+'</h3>'))
        # Adjust the slider limits based on the pixel size of the image
        x_size_selector.min, x_size_selector.max = 0, im.size[0]
        x_mod.min, x_mod.max = 0, im.size[0]
        y_size_selector.min, y_size_selector.max = 0, im.size[1]
        y_mod.min, y_mod.max = 0, im.size[1]
        y_size_selector.value = (int(im.size[1]/3), int(2*im.size[1]/3))
        x_size_selector.value = (int(im.size[0]/3), int(2*im.size[0]/3))
        
        # Instantiate shape object
        shape = SHAPE_DICT[crop_shape](x_size, y_size)
        
        # Get the axs and imshow on the existing axs
        def add_crop_patch(x_size, y_size):
            global im
            global im_draw
            
            # Ensure shape is up to date on slider changes
            shape.convert_sliders_to_shape_params(x_size, y_size)
            # Draw the shape on im_draw
            shape.draw(im_draw)
            # Plot the shape on im_draw
            shape.plot(im_draw, optimize)
            im_draw = im.copy()

        interact(add_crop_patch, x_size=x_mod, y_size = y_mod)
        
        # Click button to save params
        save_crop_sizes=interact.options(manual=True, manual_name="Save Crop Sizes")
        @save_crop_sizes
        def on_button_click(image_name=fixed(image_name)):
            """
            This will handle the onbutton click event of the cropper.
            """
            # Compute the size based on the value of x_mod and y_mod
            x_size, y_size = x_mod.value, y_mod.value
            colstart= x_size[0]
            width = x_size[1]-x_size[0]
            rowstart = im.shape[0]-y_size[1]
            height = y_size[1]-y_size[0]
            size = (colstart, rowstart, width, height)
            
            # Call the callback function
            callback(image_name, size)
            coord_dict[image_name] = size
            im_index = image_list.index(image_name)
            if im_index != len(image_list)-1:
                image_selector.value = image_list[im_index+1]
            else:
                print('Made it through all the tests; Stopping execution now.')
                sysexit()
            time.sleep(5) # Change this to change how long you have to look at the cropped results
            return
        return {'image':im, 'size':(0,0,0,0)}
    # Define test_name selector
    image_selector = widgets.Dropdown(
    options=image_list,
    value=image_list[0],
    description='Image Name:',
    disabled=False,
    continuous_update=continuous_update,
    layout=Layout(grid_area='im_selector')
    )

    # Define selectors for int range sliders; We'll define value and max, min 
    # once we know image size
    x_size_selector = widgets.IntRangeSlider(
        step=1,
        description='Width: ',
        continuous_update=continuous_update,
        orientation='horizontal',
        layout=Layout(width='auto', grid_area='width')
    )

    y_size_selector = widgets.IntRangeSlider(
        step=1,
        description='Height: ',
        continuous_update=continuous_update,
        orientation='vertical',
#         readout=True,
        layout=Layout(width='auto',height = 'auto', grid_area='height')
    )
    # Define mod sliders for cropping function; We'll define value and max, min 
    # once we know image size
    x_mod = widgets.IntRangeSlider(
        layout=Layout(width='0%',height='0%', visibility='hidden')
    )
    y_mod = widgets.IntRangeSlider(
        layout=Layout(width='0%',height='0%', visibility='hidden')
    )
    im_mod = widgets.Select(
    options=image_list,
    value=image_list[0],
    layout=Layout(width='0%',height='0%', visibility='hidden')
    )

    # Link the x,y,im selectors to mod widgets for the interactive function
    widgetLinkx = widgets.jslink((x_size_selector, 'value'), (x_mod, 'value'))
    widgetLinky = widgets.jslink((y_size_selector, 'value'), (y_mod, 'value'))
    widgetLinktn = widgets.jslink((image_selector, 'index'), (im_mod, 'index'))

    cropper = interactive(show_image, image_name =im_mod, layout=Layout(width='auto', grid_area='main'));
    return GridBox(children=[image_selector,x_size_selector,y_size_selector, cropper ],
        layout=Layout(
            width='90%',
            grid_template_rows='10% 90%',
            grid_template_columns='30% 70%',
            grid_template_areas='''
            "im_selector width"
            "height main "
            '''))

IndentationError: expected an indented block (shapes.py, line 12)

In [15]:
display('me)')
display('you')
def func():
    display('you')
    
func()

'me)'

'you'

'you'